## Key Value Operations
While Couchbase is a document database, at its heart is a distributed key-value (KV) store.
A KV store is an extremely simple, schema-less approach to data management that, as
the name implies, stores a unique ID (key) together with a piece of arbitrary information
(value); it may be thought of as a hash map or dictionary. The KV store itself can accept
any data, whether it be a binary blob or a JSON document, and Couchbase features such
as SQL++(formerly N1QL) make use of the KV store’s ability to process JSON documents.

Due to their simplicity, KV operations execute with extremely low latency, often sub-
millisecond. The KV store is accessed using simple CRUD (Create, Read, Update, Delete)
APIs, and provide the simplest interface when accessing documents using their IDs.
The KV store contains the authoritative, most up-to-date state for each item. Query, and
other services, provide eventually consistent indexes, but querying the KV store directly
will always access the latest version of data. Applications use the KV store when speed,
consistency, and simplified access patterns are preferred over flexible query options.
All KV operations are atomic, which means that Read and Update are individual operations.

In order to avoid conflicts that might arise with multiple concurrent updates to the same document, applications may make use of Compare-And-Swap (CAS), which is a per document checksum that Couchbase modifies each time a document is changed. We will see more on CAS in a later section.

Key Value (KV) or data service offers the simplest way to retrieve or mutate data where the key is known. Here we cover CRUD operations, document expiration, and optimistic locking with CAS.

### Configuring the Couchbase Cluster Information for Examples

The configuration is stored in an environment file, `.env` in this folder. 

Note that you might have to check for hidden files to see this file on Unix environments.

This file can be used to update the connection settings.
* DB_HOST: Set to `couchbase://couchbase` by default for connecting to the Couchbase cluster in the docker environment via Docker Compose. If you are running Couchbase locally on your machine via docker or installation, you can change the connection string to `couchbase://localhost`.
* DB_USER: Set to `Administrator` by default. If it is different for your cluster, please update the file.
* DB_PASSWORD: Set to `Password` by default. If it is different for your cluster, please update the file.


In [ ]:
# Read the Database information from .env file
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env file.

In [ ]:
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
print(f"Environment Settings \n{DB_HOST=} \n{DB_USER=} \n{DB_PASSWORD=}")

### Connecting to Couchbase Cluster
- Connection String: `couchbase://couchbase` would connect to the Couchbase instance.
- PasswordAuthenticator: It specifies the username & password used to access the Cluster.

#### Note
If you are running Couchbase locally on your machine via docker or installation, you can change the connection string to `couchbase://localhost` via the configuration file `.env`

In [ ]:
from couchbase.auth import PasswordAuthenticator

# needed to support SQL++ (N1QL) query
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions, QueryOptions

# get a reference to our cluster
cluster = Cluster(DB_HOST, ClusterOptions(PasswordAuthenticator(DB_USER, DB_PASSWORD)))

## Create a New Bucket in Couchbase
Go to Buckets Menu & Select `Add Bucket` on the Right.

![image](./img/Create_Bucket.png)

Note: In case creating the Bucket fails, try to allocate lower amounts of memory like 128MiB. 

### Bucket Types
- Couchbase buckets: These store data persistently, as well as in memory. They allow data to be automatically replicated for high availability, using the Database Change Protocol (DCP); and dynamically scaled across multiple clusters, by means of Cross Datacenter Replication (XDCR).
- Ephemeral buckets: These are an alternative to Couchbase buckets, to be used whenever persistence is not required: for example, when repeated disk-access involves too much overhead. This allows highly consistent in-memory performance, without disk-based fluctuations. It also allows faster node rebalances and restarts.
- Memcached buckets: These are now deprecated. Memcached buckets are designed to be used alongside other database platforms, such as ones employing relational database technology. By caching frequently-used data, Memcached buckets reduce the number of queries a database-server must perform. Each Memcached bucket provides a directly addressable, distributed, in-memory key-value cache.



In [ ]:
# get a reference to our bucket
cb = cluster.bucket("KV_Testing")

In [ ]:
# By default, there is an _default scope & collection in each bucket
cb_coll = cb.scope("_default").collection("_default")

## Insert Document

In [ ]:
document = {"foo": "bar", "bar": "foo"}
try:
    result = cb_coll.insert("document-key", document)
    cas = result.cas
    print(cas)
except Exception as e:
    print(e)

Now let us verify that the document exists in the bucket by going to the web console & checking the documents
![Check-Documents](./img/Check_Document.png)

## Fetch Document

In [ ]:
result = cb_coll.get("document-key")
doc = result.content_as[dict]
print(doc)

## Upsert Document
An upsert operation inserts the document into a collection if they do not already exist, or updates them if they do.

In [ ]:
content = {"foobar": "barfoo"}
result = cb_coll.upsert("document-key", content)

updated_doc = cb_coll.get("document-key")
upserted_doc = updated_doc.content_as[dict]
print(f"Upserted Document: {upserted_doc}")

In [ ]:
document = {"foo": "bar", "bar": "foo"}
result = cb_coll.upsert("document-key-1", document)

# fetch the new document
inserted_doc = cb_coll.get("document-key-1")
upserted_doc = inserted_doc.content_as[dict]
print(f"Upserted Document: {upserted_doc}")

## Remove Document

In [ ]:
# Remove document with document-key-1
try:
    result = cb_coll.remove("document-key-1")
except Exception as e:
    print(e)

In [ ]:
try:
    result = cb_coll.get("document-key-1")
except Exception as e:
    print(e)

## Replace Document

In [ ]:
result = cb_coll.get("document-key")
print(f"Document Before Replace: {result.content_as[dict]}")

document = {"foo": "bar", "bar": "foo"}
result = cb_coll.replace("document-key", document)

result = cb_coll.get("document-key")
print(f"Document After Replace: {result.content_as[dict]}")

## Compare and Swap (CAS) Value
The CAS is a value representing the current state of an item. Each time the item is modified, its CAS changes.

The CAS value itself is returned as part of a document’s metadata whenever a document is accessed. In the SDK, this is presented as the cas field in the result object from any operation which executes successfully.

CAS is an acronym for Compare And Swap, and is a form of optimistic locking. The CAS can be supplied as parameters to the replace and remove operations. When applications provide the CAS, server will check the application-provided version of CAS against the CAS of the document on the server:

- If the two CAS values match (they compare successfully), then the mutation operation succeeds.

- If the two CAS values differ, then the mutation operation fails.

### Durability
Writes in Couchbase are written to a single node, and from there the Couchbase Server will take care of sending that mutation to any configured replicas. The optional durability parameter, which all mutating operations accept, allows the application to wait until this replication (or persistence) is successful before proceeding.

The SDK exposes three durability levels:

- Majority - The server will ensure that the change is available in memory on the majority of configured replicas.

- MajorityAndPersistToActive - Majority level, plus persisted to disk on the active node.

- PersistToMajority - Majority level, plus persisted to disk on the majority of configured replicas.

The options are in increasing levels of safety. Note that nothing comes for free - for a given node, waiting for writes to storage is considerably slower than waiting for it to be available in-memory. 

In [ ]:
# Upsert with Durability level Majority
# The tradeoffs associated with durability levels may not be apparent in this example
# since we are using a single node cluster, but become much more clear on multi-node clusters
# The error is due to the single node setup
from couchbase.options import UpsertOptions
from couchbase.durability import Durability, ServerDurability

document = dict(foo="bar", bar="foo")
opts = UpsertOptions(durability=ServerDurability(Durability.MAJORITY))
try:
    result = cb_coll.upsert("document-key", document, opts)
except Exception as e:
    print(e)

## Document Expiration
In Couchbase, documents can be set to expire after a specified amount of time. They can be specified at the time of creating a document. 

The maximum value for TTL is 2147483648 seconds  or 68.096 years. The default value is 0, which indicates that TTL is disabled. If TTL is changed from the default, it is thereby enabled.

When the expiration time is reached, Couchbase Server deletes the item the next time it is accessed or when the cleanup process is run periodically. Following the deletion, a tombstone is maintained by Couchbase Server, as a record.

A tombstone is a record of an item that has been removed. Tombstones are maintained in order to provide eventual consistency, between nodes and between clusters. They are removed when the metadata is cleaned up by a process known as Metadata Purge which removes all the references to the delted data.

In [ ]:
# Insert document with expiry option
import time
from datetime import timedelta

from couchbase.options import GetOptions, InsertOptions

document = {"foo": "bar", "bar": "foo"}
opts = InsertOptions(timeout=timedelta(seconds=5))
result = cb_coll.insert(
    "document-key-opts", document, opts, expiry=timedelta(seconds=20)
)
res = cb_coll.get("document-key-opts")
print(f"Inserted Document: {res.content_as[dict]}")

# Sleep for 20 seconds
time.sleep(20)

# Extend expiry for a document
extend = cb_coll.touch("document-key-opts", timedelta(seconds=10))

expiry = cb_coll.get("document-key-opts", GetOptions(with_expiry=True))
print(f"Expiry of Document: {expiry.expiryTime}")

Verify that the document is deleted after the expiry time is reached. You can see the tombstone record here which will be deleted after a while.

![Expire-Key](./img/Expire_Key.png)

## Use Cases for KV - Caching
Caching is a process by which data is stored in a temporary location so that they can be accessed faster in the future.

The KV operations can be used for caching data that is frequently accessed by applications. The expiry options can be used to set limited time documents which is quite common with caching. 

You can go through the following example in case you want to get a complete example of Caching.

[Full Example using a Flask server for Caching](https://docs.couchbase.com/python-sdk/current/howtos/caching-example.html)

## Sub-Document Operations
Sub-Document operations can be used to efficiently access and change parts of documents.

While full-document retrievals retrieve the entire document and full document updates require sending the entire document, Sub-Document retrievals only retrieve relevant parts of a document and Sub-Document updates only require sending the updated portions of a document.

Sub-Document operations are also atomic, in that if one Sub-Document mutation fails then all will, allowing safe modifications to documents with built-in concurrency control.

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=4, depth=6)

In [ ]:
document = {
    "name": "Douglas Reynholm",
    "email": "douglas@reynholmindustries.com",
    "addresses": {
        "billing": {
            "line1": "123 Any Street",
            "line2": "Anytown",
            "country": "United Kingdom",
        },
        "delivery": {
            "line1": "123 Any Street",
            "line2": "Anytown",
            "country": "United Kingdom",
        },
    },
    "purchases": {"complete": [339, 976, 442, 666], "abandoned": [157, 42, 999]},
}

## Exercise 2.1
Insert this Document into the KV store with the Key "customer123". 
You need to finish this exercise for the following two code samples to work.

In [ ]:
# Solution


In [ ]:
# Sub-Document Lookup / Fetch sub-document inside the document
import couchbase.subdocument as SD

try:
    result = cb_coll.lookup_in("customer123", [SD.get("addresses.delivery.country")])
    country = result.content_as[str](0)
    print(country)
except Exceptiona as e:
    print(e)

In [ ]:
# Sub-Document Check / Check for existence of sub-document inside the document
try:
    result = cb_coll.lookup_in("customer123", [SD.exists("purchases.pending[-1]")])
    print(f"Path exists: {result.exists(0)}")
except Exception as e:
    print(e)

## Exercise 2.2
1. Check for the existence of abandoned purchases for the customer

2. Get the list of completed purchases for the customer

3. [Bonus] Do the two operations in a single SDK operation 

In [ ]:
# Solution 1


In [ ]:
# Solution 2


In [ ]:
# Solution 3


## Mutate Sub-Documents

In [ ]:
# Insert Sub-Document
cb_coll.mutate_in("customer123", [SD.upsert("fax", "311-555-0151")])

result = cb_coll.get("customer123")
pp.pprint(result.content_as[dict])

In [ ]:
# You cannot insert sub-document into an existing path
from couchbase.exceptions import PathExistsException

try:
    cb_coll.mutate_in(
        "customer123", [SD.insert("purchases.complete", [42, True, "None"])]
    )
except PathExistsException:
    print("Path exists, cannot use insert.")

In [ ]:
result = cb_coll.get("customer123")
pp.pprint(result.content_as[dict])

In [ ]:
# Modify Arrays in Sub-Documents using array_append / array_prepend
cb_coll.mutate_in(
    "customer123",
    (
        SD.array_append("purchases.complete", 777),
        SD.array_prepend("purchases.abandoned", 18),
    ),
)
result = cb_coll.get("customer123")
pp.pprint(result.content_as[dict])

## KV Transactions
Couchbase also supports distributed transactions using the Python SDK. 

You can read more about how to perform transactions [here ](https://docs.couchbase.com/python-sdk/current/howtos/distributed-acid-transactions-from-the-sdk.html)

## Exercise 2.3
1. Fetch the details of the airline with the key `airline_1355`
2. Get the altitude of the airport with key `airport_1309`
3. Check for the existence of "schedule" for the route with key `route_10006`
4. Get the first rating for "Sleep Quality" for the hotel with key `hotel_10160`

## Solutions

In [ ]:
# Solution 1


In [ ]:
# Solution 2


In [ ]:
# Solution 3


In [ ]:
# Solution 4


## References
- [Key Value Operations](https://docs.couchbase.com/python-sdk/current/howtos/kv-operations.html)
- [Sub-Document Operations](https://docs.couchbase.com/python-sdk/current/howtos/subdocument-operations.html)
- [Python Transactions](https://docs.couchbase.com/python-sdk/current/howtos/distributed-acid-transactions-from-the-sdk.html)